In [110]:
import requests as rq
import json
import pandas as pd
from bs4 import BeautifulSoup
import time
import os
import datetime
from dotenv import load_dotenv

In [111]:
# %pip install python-dotenv

In [112]:
# named_tuple = time.localtime() # get struct_time
# time_string = time.strftime("%dD%mM%YY", named_tuple)
# print(time.strftime("%dD", named_tuple))
# print(time_string)
today = datetime.datetime.now()
yesterday = today - datetime.timedelta(days=1)
time_string = today.strftime("%dD%mM%YY")
ytime_string = yesterday.strftime("%dD%mM%YY")
print(time_string , ytime_string)


07D05M2024Y 06D05M2024Y


In [113]:
load_dotenv()
my_path = os.getcwd()
api_key = os.getenv('API_KEY')
ninja_api_key = os.getenv('NINJA_API_KEY')
'''
Title and abstract
Document classification codes
Date of publication
Affiliations
Citations / references
Keywords
'''

# setup the HTTP headers
header = {"Accept": "application/json", "X-ELS-APIKey": api_key}
header2 = {"Accept": "*/xml", "X-ELS-APIKey": api_key}
header3 = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"}

# Break down the variables for the search query
api_url = "https://api.elsevier.com/content/search/scopus?query="
pubyear = ['2018','2019','2020','2021','2022','2023','2024']



#create folder a day
day_folder = my_path+'\\research_csv\\'+time_string
yesterday_folder = my_path+'\\research_csv\\'+ytime_string

if not os.path.exists(day_folder):
    os.mkdir(day_folder)    

#yesterday paper info and yesterday ref info
ypaper_info = pd.DataFrame()
yref_info = pd.DataFrame()

if os.path.exists(yesterday_folder):
    if os.path.exists(yesterday_folder+'\\paper_info.csv'):
        ypaper_info = pd.read_csv(yesterday_folder+'\\paper_info.csv')
    if os.path.exists(yesterday_folder+'\\ref_info.csv'):
        yref_info = pd.read_csv(yesterday_folder+'\\ref_info.csv')

#paper next id and ref next id
paperNid = len(ypaper_info.index)
print(paperNid)


1400


In [114]:
dt = dict()
dt['id'] = []
dt['link']= []
dt['title']= []
dt['publicationName']= []
dt['affiliation']= []
dt['abstract'] = []
dt['publicationDate'] = []
dt['ref_count'] = []
dt['publicationYear'] = []


ref = dict()
ref['id'] = []
ref['ref_authors'] = []
ref['ref_publicationyear'] = []
ref['ref_title'] = []





def to_csv(result,dt,ref):
    oldlen = len(dt['id'])
    for id,paper in enumerate(result['search-results']['entry']):
        Nid = paperNid + id + oldlen
        dt['id'].append(Nid)
        dt['link'].append(paper['link'][2]['@href'])
        dt['title'].append(paper['dc:title'])
        dt['publicationName'].append(paper['prism:publicationName'])
        affslist = []
        if 'affiliation' in paper:
            affs = paper['affiliation']
            for aff in affs:
                affslist.append((aff['affilname'] , aff['affiliation-city'] , aff['affiliation-country']))
        dt['affiliation'].append(affslist)
        
        # print(paper['link'][2]['@href'])
        page = rq.get(paper['link'][0]['@href'],headers=header)
        result_ab = page.json()
        #get abstract
        ab = result_ab['abstracts-retrieval-response']['item']['bibrecord']['head']['abstracts']
        #get publication date
        pubdate = result_ab['abstracts-retrieval-response']['item']['bibrecord']['head']['source']['publicationdate']
        dt['abstract'].append(ab)
        dt['publicationDate'].append(pubdate)
        dt['publicationYear'].append(pubdate['year'])
        # print(id,result_ab)
        # get reference count
        if result_ab['abstracts-retrieval-response']['item']['bibrecord']['tail'] != None :
            refcount = result_ab['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['@refcount']
            dt['ref_count'].append(refcount)
        else:
            dt['ref_count'].append('null')
    
        #reference paper csv
        if result_ab['abstracts-retrieval-response']['item']['bibrecord']['tail'] != None:
            allref = result_ab['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference']
            for j in allref:
                # print(Nid , j)
                if type(j) is dict:
                    ref['id'].append(Nid)
                    if 'ref-fulltext' in j:
                        ref['ref_authors'].append(j['ref-fulltext'])
                    else:
                        ref['ref_authors'].append('null')
                    if "ref-publicationyear" in j['ref-info']:
                        ref['ref_publicationyear'].append(j['ref-info']['ref-publicationyear']['@first'])
                    else:
                        ref['ref_publicationyear'].append('null')
                    if 'ref-title' in j['ref-info']:
                        ref['ref_title'].append(j['ref-info']['ref-title']['ref-titletext'])
                    else:
                        ref['ref_title'].append('null')
        # break

        # df.to_csv(my_path+"\\research_csv\\paper_info\\"+"scopus_paper_"+str(id)+'.csv')

    return dt,ref


In [115]:
#publicatino year 2018 and doctype article
days_fetch = len(os.listdir('.\\research_csv'))-1
for i in range(len(pubyear)):
    search_query = '%28PUBYEAR%20%3D%20'+pubyear[i]+'%29%20AND%20%28DOCTYPE%28ar%29%29'
    itemperpage = '200'
    response = rq.get(api_url + search_query +"&count="+itemperpage  + "&start=" +str(int(itemperpage)*days_fetch) , headers = header)
    result = (response.json())
    # print(result)
    dt, ref = to_csv(result,dt,ref)
    
df = pd.DataFrame.from_dict(dt)
ref_df = pd.DataFrame.from_dict(ref)

df = pd.concat([ypaper_info , df] , join='inner' ,ignore_index=True)
ref_df = pd.concat([yref_info , ref_df] , join='inner' , ignore_index=True)

df.to_csv(day_folder+'\\paper_info.csv')
ref_df.to_csv(day_folder+'\\ref_info.csv')

In [138]:
df = pd.DataFrame.from_dict(dt)
ref_df = pd.DataFrame.from_dict(ref)

df = pd.concat([ypaper_info , df] , join='inner' ,ignore_index=True)
ref_df = pd.concat([yref_info , ref_df] , join='inner' , ignore_index=True)

df.to_csv(day_folder+'\\paper_info.csv')
ref_df.to_csv(day_folder+'\\ref_info.csv')



In [116]:
# df.to_csv(yesterday_folder+'\\paper_info.csv')
# ref_df.to_csv(yesterday_folder+'\\ref_info.csv')

In [117]:
# print(result)
# paper_info = pd.read_csv(my_path+'\\research_csv\\'+ time_string +'\\paper_info.csv')
# paper_info['affil_coor'] = ''
# coor = dict()
# if os.path.exists(my_path+'\\coor.json'):
#     f = open('coor.json')
#     coor = json.load(f)
#     f.close()
    

# for i in range(1,len(paper_info.index)):
#     aff = paper_info.loc[i,'affiliation']
#     if aff != 'nan':
#         print(aff)
#         afflist = eval(aff)
#         for j in afflist:
#             city = j['affiliation-city']
#             country = j['affiliation-country']
#             coor_key = city + ' ' + country
#             if coor_key not in coor:
#                 api_url = 'https://api.api-ninjas.com/v1/geocoding?city='
#                 response = rq.get(api_url + city + '&country=' + country, headers={'X-Api-Key': ninja_api_key})
#                 resjson = response.json()
#                 if 'error' not in resjson:
#                     coor[coor_key] = str(resjson[0]['latitude']) + " " + str(resjson[0]['longitude'])
#                     paper_info.loc[i,'affil_coor'] = coor[coor_key]
#                     print(city,country,resjson,coor)
#             else:
#                 paper_info.loc[i,'affil_coor'] = coor[coor_key]



In [118]:
# with open("coor.json",'w') as coorjson:
#     json.dump(coor , coorjson)

In [119]:
# save_file = open("savedata2.json", "w")  
# json.dump(result_ab, save_file, indent = 6)  
# save_file.close()  


In [120]:
# corr = {'asdf':'asdf' , 'xx':'d'}
# with open("coor.json",'w') as corrjson:
#     json.dump(corr , corrjson)

In [121]:
# f = open(my_path+'\\coor.json')
# data = json.load(f)
# data['adsf'] = 'ddd'
# data['dd'] = 'x'
# f.close()

In [122]:
# a  = pd.DataFrame({'asdf':['asdf','sd','a'] , 'xx':['as','d','p']})
# a['ff'] = ""
# a.loc[0,'ff'] = "78,45"
# a
# df['publicationDate'][0]

In [123]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# import time
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# import os
# # Set the download directory for the browser
# # C:\\Users\\acer\\Desktop\\code\\DataSci&DataEng\\project
# download_directory = my_path+"\\publication_recommend\\webscraping\\research_csv"
# chrome_options = webdriver.ChromeOptions()
# prefs = {"download.default_directory": download_directory}
# chrome_options.add_experimental_option("prefs", prefs)
# browser = webdriver.Chrome(options = chrome_options)
# wait = WebDriverWait(browser, 10)
# url = str(link)
# browser.get(url=url)
# time.sleep(2)
# for id,i in enumerate(result['search-results']['entry']):
#         filename = 'scopus'+str(id)
#         os.mkdir(download_directory+'\\'+filename)
#         link = i['link'][2]['@href']
#         url = str(link)
#         browser.get(url=url)
#         time.sleep(2)
#         #download research info
#         browser.find_element(By.XPATH,"//div[@class = 'Stack-module__tT3r4 Stack-module__Y4rmW']//div[@class='export-dropdown']").click()
#         browser.find_element(By.XPATH,"//div[@class = 'Stack-module__tT3r4 Stack-module___CTfk']//button[@data-testid='export-to-csv']").click()
#         browser.find_element(By.XPATH,"//div[@class = 'Stack-module__tT3r4 Stack-module___CTfk'][2]//label[@class='Checkbox-module__jE3jb Checkbox-module__zALPa']").click()
#         browser.find_element(By.XPATH,"//div[@class = 'Stack-module__tT3r4 Stack-module___CTfk'][3]//label[@class='Checkbox-module__jE3jb Checkbox-module__zALPa']").click()
#         browser.find_element(By.XPATH,"//button[@data-testid='submit-export-button']").click()
#         #dowload research reference info
#         time.sleep(3)
#         browser.find_element(By.XPATH,"//div[@id='resultsListToolbar']//label[@for='referencePageCheckbox']").click()
#         browser.find_element(By.XPATH,"//button[@id='export_references']").click()
#         time.sleep(1)
#         browser.find_element(By.XPATH,"//ul[@id='exportList']//li[@class='radio-inline valignTop'][5]").click()
#         browser.find_element(By.XPATH,"//button[(@class = 'btn btn-primary btn-sm btnEnable') and (@id = 'exportTrigger')]").click()

In [124]:
ref_df

,Unnamed: 0,id,ref_authors,ref_publicationyear,ref_title
0,0.0,0,European Commission (2015) Energy Union Packag...,2015.0,NaN
1,1.0,0,European Commission (2017) White Paper on the ...,2017.0,NaN
2,2.0,1,"Barthes, Roland. 2005. The Language of Fashion...",2005.0,NaN
3,3.0,1,"Bojović, Brankica. 2010. Leksika u kriminološk...",2010.0,NaN
4,4.0,1,"Bojović, Brankica. 2013. “Pictorial Language i...",2013.0,NaN
...,...,...,...,...,...
51320,NaN,2799,A. Sharma et al. The skin as a metabolic and i...,2019,The skin as a metabolic and immune-competent o...
51321,NaN,2799,J. Line E. Saville X. Meng D. Naisbitt Why dru...,2023,Why drug exposure is frequently associated wit...
51322,NaN,2799,J. Strid S. Strobel Skin barrier dysfunction a...,2005,Skin barrier dysfunction and systemic sensitiz...
51323,NaN,2799,M.D. Ramírez-González M. Herrera-Enríquez L.G....,2009,Role of epidermal dendritic cells in drug-indu...


In [125]:
a = {'a':[1] , 'b':[2] , 'c':[3]}
adf = pd.DataFrame.from_dict(a)

b = {'a':[12] , 'b':[33] , 'c':[31]}
bdf = pd.DataFrame.from_dict(b)

cdf = pd.concat([adf,bdf])

cdf


,a,b,c
0,1,2,3
0,12,33,31


In [126]:
df

,Unnamed: 0,id,link,title,publicationName,affiliation,abstract,publicationDate,ref_count,publicationYear
0,0.0,0,https://www.scopus.com/inward/record.uri?partn...,Social Progress for Resilient Regions,Region,"[('European Investment Bank', 'Luxembourg', 'L...","© 2018, European Regional Science Association....","{'month': '12', 'year': '2018', 'date-text': {...",2.0,2018
1,1.0,1,https://www.scopus.com/inward/record.uri?partn...,Fashion language and translatology,Babel,"[('University of Montenegro', 'Podgorica', 'Mo...",© John Benjamins Publishing Company.My study i...,"{'month': '12', 'year': '2018', 'date-text': {...",18.0,2018
2,2.0,2,https://www.scopus.com/inward/record.uri?partn...,A pragmatic framework to note-taking in consec...,Babel,"[('An-Najah National University', 'Nablus', 'P...",© John Benjamins Publishing Company.This study...,"{'month': '12', 'year': '2018', 'date-text': {...",17.0,2018
3,3.0,3,https://www.scopus.com/inward/record.uri?partn...,An Analytic Approximation to the Density of Tw...,Recoletos Multidisciplinary Research Journal,[('Agusan del Sur State College of Agriculture...,"© 2018, University of San Jose-Recoletos. All ...","{'month': '12', 'year': '2018', 'date-text': {...",11.0,2018
4,4.0,4,https://www.scopus.com/inward/record.uri?partn...,DANCE MOTIFS ON PREHISTORIC POTTERY FROM EASTE...,Vjesnik Arheoloskog Muzeja u Zagrebu,"[('Centre for Experimental Archaeology', 'Zagr...","© 2018, Archaeological museum in Zagreb. All r...","{'month': '12', 'year': '2018', 'date-text': {...",37.0,2018
...,...,...,...,...,...,...,...,...,...,...
1395,NaN,2795,https://www.scopus.com/inward/record.uri?partn...,Evaluating Night-Eating Syndrome in bariatric ...,BMC Research Notes,"[(King Abdulaziz University, Jeddah, Saudi Ara...",© The Author(s) 2024.Background: Night-Eating ...,"{'month': '12', 'year': '2024', 'date-text': '...",25,2024
1396,NaN,2796,https://www.scopus.com/inward/record.uri?partn...,Cardiovascular mortality risk in patients with...,Journal of Ovarian Research,"[(Dalian Medical University, Dalian, China)]",© The Author(s) 2024.Objectives: Ovarian cance...,"{'month': '12', 'year': '2024', 'date-text': '...",32,2024
1397,NaN,2797,https://www.scopus.com/inward/record.uri?partn...,EZH2-dependent epigenetic reprogramming in the...,Translational Psychiatry,"[(University of Illinois at Chicago, Chicago, ...",© This is a U.S. Government work and not under...,"{'month': '12', 'year': '2024', 'date-text': '...",62,2024
1398,NaN,2798,https://www.scopus.com/inward/record.uri?partn...,Exploring medical and nursing students’ percep...,BMC Medical Education,"[(The Aga Khan University, Karachi, Pakistan)]",© The Author(s) 2024.Background: Educating hea...,"{'month': '12', 'year': '2024', 'date-text': '...",56,2024
